In [4]:
import streamlit as st
import pandas as pd
import plotly.express as px
# from crash_cost_analysis import run_crash_analysis
from initial_pert import run_pert_analysis 
from completion_probability import show_completion_probability_ui
from plot_aoa_diagram import plot_aoa_network
# # from plot_aoa import plot_aoa_network

# import sys
# import os
# # sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "scripts")))
# # import sys
# # import os
# sys.path.append(os.path.dirname(__file__))




st.title("📊 PERT Scheduler + Crash Cost Analyzer")

uploaded_file = st.file_uploader("📥 Upload Cleaned Excel", type=["xlsx"])

if uploaded_file:
    df = pd.read_excel(uploaded_file)
    df.columns = df.columns.str.strip()

    # Rename for consistency
    df = df.rename(columns={
        "Most Likely": "MostLikely",
        "Optimistic Time": "Optimistic",
        "Pessimistic Time": "Pessimistic",
        "Predecessors": "Dependencies",
        "Activity": "Activity_id",
        "Activity Name": "Activity N",

    })

    st.subheader("🧾 Raw Data")
    st.dataframe(df)

    if st.button("🚀 Run PERT Analysis"):
        result_df, critical_path, G = run_pert_analysis(df)      # debug 1
        st.success("PERT Analysis Complete ✅")
        st.write("🔴 Critical Path:", " → ".join(critical_path))   # maybe
        st.dataframe(result_df)



ModuleNotFoundError: No module named 'initial_pert'

In [3]:
import networkx as nx
import pandas as pd

def run_pert_analysis(df):
    

    # Calculate Expected Time using PERT formula
    df["ExpectedTime"] = (df["Optimistic"] + 4 * df["MostLikely"] + df["Pessimistic"]) / 6
    df["Variance"] = ((df["Pessimistic"] - df["Optimistic"]) / 6) ** 2

    # Initialize graph
    G = nx.DiGraph()

    # Add nodes with duration
    for _, row in df.iterrows():
        act_id = row["Activity_id"]
        label = f"{act_id} - {row['Activity N']}"
        G.add_node(label, duration=row["ExpectedTime"])

    # Add edges based on dependencies
    for _, row in df.iterrows():
        current = f"{row['Activity_id']} - {row['Activity N']}"
        if pd.notna(row["Dependencies"]):
            deps = [d.strip() for d in str(row["Dependencies"]).split(',')]
            for dep in deps:
                dep_name = df[df["Activity_id"] == dep]["Activity N"].values
                if len(dep_name) == 0:
                    continue
                dep_full = f"{dep} - {dep_name[0]}"
                G.add_edge(dep_full, current)

    # Forward pass (Early Start and Finish)
    ES, EF = {}, {}
    for node in nx.topological_sort(G):
        preds = list(G.predecessors(node))
        ES[node] = max([EF[p] for p in preds], default=0)
        EF[node] = ES[node] + G.nodes[node]['duration']
        G.nodes[node]['ES'] = ES[node]
        G.nodes[node]['EF'] = EF[node]

    # Backward pass (Late Start and Finish)
    LS, LF = {}, {}
    max_EF = max(EF.values())
    for node in reversed(list(nx.topological_sort(G))):
        succs = list(G.successors(node))
        LF[node] = min([LS[s] for s in succs], default=max_EF)
        LS[node] = LF[node] - G.nodes[node]['duration']
        G.nodes[node]['LS'] = LS[node]
        G.nodes[node]['LF'] = LF[node]
        G.nodes[node]['Slack'] = LS[node] - ES[node]

    # Critical Path
    critical_path = [node for node in G.nodes if G.nodes[node]['Slack'] == 0]

    # Convert to result dataframe
    result_data = []
    for node in G.nodes:
        result_data.append({
            "Activity N": node,
            "Duration": G.nodes[node]['duration'],
            "ES": G.nodes[node]['ES'],
            "EF": G.nodes[node]['EF'],
            "LS": G.nodes[node]['LS'],
            "LF": G.nodes[node]['LF'],
            "Slack": G.nodes[node]['Slack'],
            "IsCritical": node in critical_path
        })
    result_df = pd.DataFrame(result_data)

    return result_df, critical_path, G
